# Detector de Texto para OCR
Este modelo pretende detectar la resencia de texto en una imagen y en lo posible retornar las lineas de texto y cajas de los carcteres que las componen.

Modelo basado en yolo 8

## Importación de Librerias

In [1]:
#!pip install -q ultralytics
from ultralytics import YOLO

In [2]:
#Importo librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import cv2
import random
from PIL import Image, ImageFilter, ImageDraw, ImageFont
from IPython.display import Image as ImageDisplay
from IPython.core.display import HTML
from io import BytesIO
import base64
from sklearn.model_selection import train_test_split

## Generacion de imagenes

In [ ]:
#Descomprimo las fuentes

#!unzip c:/tmp/content/fuentes.zip -d c:/tmp/content/

"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Funciones auxiliares

In [6]:
# Funciones auxiliares
def np_to_base64_img(arr):
    """Convierte un np.array a una imagen embebida en base64"""
    h,w = arr.shape[:2]
    im = Image.fromarray(arr)
    buffer = BytesIO()
    im.save(buffer, format="PNG")
    img_str = base64.b64encode(buffer.getvalue()).decode()
    return f'<img src="data:image/png;base64,{img_str}" width="{w}" height="{h}"/>'

In [7]:
def get_bbox_from_image(image_array):
    """
    Toma un array NumPy de una imagen en escala de grises, encuentra los contornos,
    los combina y devuelve las coordenadas de la bounding box en formato
    (x_min, y_min, x_max, y_max).

    Args:
        image_array (np.ndarray): Una imagen en escala de grises como un array NumPy.

    Returns:
        tuple: Una tupla (x_min, y_min, x_max, y_max) que representa la bounding box.
               Retorna None si no se encuentran contornos.
    """
    # Asegurarse de que la imagen esté en escala de grises si no lo está ya
    if len(image_array.shape) == 3:
        gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image_array

    # Binarizar la imagen si es necesario para asegurar buenos contornos
    # Esto es crucial si la imagen de entrada tiene tonos de gris en lugar de solo blanco/negro
    # Ajusta el umbral (e.g., 127) y el tipo (e.g., cv2.THRESH_BINARY_INV) según tus imágenes.
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)


    # Encontrar contornos
    # cv2.RETR_EXTERNAL recupera solo los contornos externos (útil para caracteres con agujeros internos)
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        binary_image=cv2.bitwise_not(binary_image)
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None # No se encontraron contornos

    # Combinar todos los contornos en uno solo (para caracteres con múltiples partes como 'i' o 'j')
    # Solo apilamos si hay contornos válidos (size > 0)
    valid_contours = [c.squeeze() for c in contours if c.size > 0]
    if not valid_contours:
        return None # No se encontraron contornos válidos después de limpiar

    all_points = np.vstack(valid_contours)

    # Encontrar el rectángulo delimitador para el contorno combinado
    x, y, w, h = cv2.boundingRect(all_points)

    # Retornar en el formato (x_min, y_min, x_max, y_max)
    x_min, y_min = x, y
    x_max, y_max = x + w, y + h

    return (x_min, y_min, x_max, y_max)

In [ ]:
def generate_combinations(char_list_original, min_len=3, max_len=7):

    # 1. Convertir la cadena original a una lista y desordenarla una única vez
    shuffled_char_pool = list(char_list_original)
    random.shuffle(shuffled_char_pool)

    generated_combinations = []

    pool_index = 0
    total_pool_len = len(shuffled_char_pool)

    while pool_index < total_pool_len:
        # Calcular cuántos caracteres quedan en el pool
        remaining_chars = total_pool_len - pool_index

        # Determinar la longitud de la combinación actual
        # Si quedan 7 o menos caracteres (o menos de max_len) y al menos min_len, usa todos los restantes.
        # De lo contrario, elige una longitud aleatoria entre min_len y max_len.
        if remaining_chars <= max_len:
            current_len = remaining_chars
        else:
            current_len = random.randint(min_len, max_len)
            # Asegurarse de que la longitud elegida no exceda los caracteres restantes
            current_len = min(current_len, remaining_chars)

        # Extraer la combinación del pool
        combination_chars = shuffled_char_pool[pool_index : pool_index + current_len]
        combination = "".join(combination_chars)

        generated_combinations.append(combination)
        pool_index += current_len # Mover el índice al siguiente punto en el pool

    print(f"Número de combinaciones generadas: {len(generated_combinations)}")
    return generated_combinations

## Generacion del dataset

In [8]:
#Creo un dataset con las fuentes
def create_font_dataset(base_folder):
    """
    Crea un DataFrame de Pandas con información sobre las fuentes TTF encontradas.

    Args:
        base_folder (str): Ruta a la carpeta principal que contiene las subcarpetas de tipos de letra.

    Returns:
        pd.DataFrame: DataFrame con columnas 'font_name' y 'font_path'.
    """
    font_data = []
    print(f"Buscando archivos .ttf en: {base_folder}")
    for font_type_folder in os.listdir(base_folder):
        font_type_path = os.path.join(base_folder, font_type_folder)
        if not os.path.isdir(font_type_path):
            continue

        ttf_files = [f for f in os.listdir(font_type_path) if f.lower().endswith('.ttf')]

        if not ttf_files:
            print(f"Advertencia: No se encontraron archivos .ttf en '{font_type_path}'. Saltando.")
            continue

        # Asumimos un único archivo .ttf por carpeta de tipo de letra o elegimos el primero
        font_file_name = ttf_files[0]
        full_font_path = os.path.join(font_type_path, font_file_name)

        font_data.append({
            'font_name': font_type_folder, # Usamos el nombre de la carpeta como nombre de la fuente
            'font_path': full_font_path
        })

    return pd.DataFrame(font_data)

In [9]:
# prompt: quiero una funcion que reciba una imagen de cv en escala de grises y le aplique blur gausieano y la devuelva en el mismo formato

def apply_gaussian_blur(image_gray, ksize=(5, 5), sigmaX=0):
    """
    Aplica un filtro de desenfoque Gaussiano a una imagen en escala de grises.

    Args:
        image_gray (np.ndarray): Imagen de entrada en escala de grises (array NumPy).
        ksize (tuple): Tamaño del kernel Gaussiano. Debe ser una tupla de números
                       enteros impares positivos (ancho, alto).
        sigmaX (float): Desviación estándar del kernel en la dirección X. Si es 0,
                        se calcula automáticamente en función del tamaño del kernel.

    Returns:
        np.ndarray: La imagen desenfocada como un array NumPy en escala de grises.
    """
    if len(image_gray.shape) != 2:
        print("Advertencia: La imagen de entrada no está en escala de grises. Intentando convertir.")
        image_gray = cv2.cvtColor(image_gray, cv2.COLOR_BGR2GRAY)

    blurred_image = cv2.GaussianBlur(image_gray, ksize, sigmaX)
    return blurred_image


In [10]:
# prompt: quiero una funcion que reciba una imagen de opencv en escala de grises y la rote 5 o -5 grados y devuelva en el mismo formato

def rotate_image(image_gray, angle, color=(0, 0, 0)):
    if len(image_gray.shape) != 2:
        print("Advertencia: La imagen de entrada no está en escala de grises. Intentando convertir.")
        image_gray = cv2.cvtColor(image_gray, cv2.COLOR_BGR2GRAY)

    (h, w) = image_gray.shape[:2]
    center = (w // 2, h // 2)

    # Calcular la matriz de rotación
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Calcular las nuevas dimensiones de la imagen para evitar recortes
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # Ajustar la matriz de translación para que el centro permanezca en el centro de la nueva imagen
    M[0, 2] += (nW / 2) - center[0]
    M[1, 2] += (nH / 2) - center[1]

    # Realizar la rotación, manteniendo el fondo negro (0)
    rotated_image = cv2.warpAffine(image_gray, M, (nW, nH), borderValue=color)

    return rotated_image


In [20]:
def create_noise_background(image_shape, gray_center=128, noise_amplitude=50):
    """
    Crea un fondo de ruido aleatorio predominantemente gris, con cierta variación.

    Args:
        image_shape (tuple): Las dimensiones (altura, anchura) de la imagen.
        gray_center (int): El valor central de gris para el ruido (0-255).
        noise_amplitude (int): La amplitud del ruido alrededor del centro.
                               Por ejemplo, 50 significa ruido entre [centro-50, centro+50].

    Returns:
        np.ndarray: Un array NumPy que representa el fondo de ruido.
    """
    # Genera ruido en un rango limitado alrededor del centro
    # np.random.normal para un ruido más natural (gaussiano) o randint para uniforme.
    # Usaremos randint para simplificar y controlar el rango.

    # Calcular el rango min y max para el ruido
    min_val = max(0, gray_center - noise_amplitude)
    max_val = min(255, gray_center + noise_amplitude)

    # Genera el ruido y lo escala al rango deseado
    noise = np.random.randint(min_val, max_val + 1, image_shape, dtype=np.uint8)
    return noise

def get_bbox_from_image_array(image_array):
    """
    Toma un array NumPy de una imagen en escala de grises, encuentra los contornos,
    los combina y devuelve las coordenadas de la bounding box en formato
    (x_min, y_min, x_max, y_max).

    Args:
        image_array (np.ndarray): Una imagen en escala de grises como un array NumPy.

    Returns:
        tuple: Una tupla (x_min, y_min, x_max, y_max) que representa la bounding box.
               Retorna None si no se encuentran contornos.
    """
    # Asegurarse de que la imagen esté en escala de grises si no lo está ya
    if len(image_array.shape) == 3:
        gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image_array

    # Binarizar la imagen si es necesario para asegurar buenos contornos
    # Esto es crucial si la imagen de entrada tiene tonos de gris en lugar de solo blanco/negro
    # Ajusta el umbral (e.g., 127) y el tipo (e.g., cv2.THRESH_BINARY_INV) según tus imágenes.
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)


    # Encontrar contornos
    # cv2.RETR_EXTERNAL recupera solo los contornos externos (útil para caracteres con agujeros internos)
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        binary_image=cv2.bitwise_not(binary_image)
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None # No se encontraron contornos

    # Combinar todos los contornos en uno solo (para caracteres con múltiples partes como 'i' o 'j')
    # Solo apilamos si hay contornos válidos (size > 0)
    valid_contours = [c.squeeze() for c in contours if c.size > 0]
    if not valid_contours:
        return None # No se encontraron contornos válidos después de limpiar

    all_points = np.vstack(valid_contours)

    # Encontrar el rectángulo delimitador para el contorno combinado
    x, y, w, h = cv2.boundingRect(all_points)

    # Retornar en el formato (x_min, y_min, x_max, y_max)
    x_min, y_min = x, y
    x_max, y_max = x + w, y + h

    return (x_min, y_min, x_max, y_max)


# --- Generación de Imagen Base y Bbox (con Pillow) ---
def generate_base_char_image_and_bbox(char, font_path, font_size=200, img_size=(256, 256)):
    """
    Genera la imagen base de un caracter (blanco sobre fondo negro) usando Pillow
    y calcula su bounding box.
    """
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"Advertencia: No se pudo cargar la fuente {font_path}. Retornando None.")
        return None, None
    except Exception as e:
        print(f"Error al cargar la fuente {font_path}: {e}. Retornando None.")
        return None, None

    img_black_bg_white_char_pil = Image.new('L', img_size, color=0)
    draw = ImageDraw.Draw(img_black_bg_white_char_pil)

    try:
        # textbbox returns (left, top, right, bottom) of the text
        text_bbox = draw.textbbox((0,0), char, font=font)
    except Exception as e:
        print(f"Error al obtener textbbox para '{char}' con {font_path}: {e}. Retornando None.")
        return None, None

    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Calculate position to center the text
    # x_center: (image_width - text_width) / 2 - text_bbox[0]
    # y_center: (image_height - text_height) / 2 - text_bbox[1]
    # These calculations adjust for the actual content's bbox offset
    x_center = (img_size[0] - text_width) / 2 - text_bbox[0]
    y_center = (img_size[1] - text_height) / 2 - text_bbox[1]

    draw.text((x_center, y_center), char, font=font, fill=255)
    img_np = np.array(img_black_bg_white_char_pil)

    bbox = get_bbox_from_image_array(img_np)

    return img_np, bbox

    # Crear imagen con caracter blanco sobre fondo negro
    img_black_bg_white_char_pil = Image.new('L', img_size, color=0) # Fondo negro
    draw = ImageDraw.Draw(img_black_bg_white_char_pil)

    # Calcular posición para centrar el texto
    # Usar textbbox para obtener dimensiones del texto renderizado
    try:
        text_bbox = draw.textbbox((0,0), char, font=font)
    except Exception as e:
        print(f"Error al obtener textbbox para '{char}' con {font_path}: {e}. Retornando None.")
        return None, None

    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    x_center = (img_size[0] - text_width) / 2 - text_bbox[0]
    y_center = (img_size[1] - text_height) / 2 - text_bbox[1]

    draw.text((x_center, y_center), char, font=font, fill=255) # Caracter blanco
    img_np = np.array(img_black_bg_white_char_pil) # Convertir a NumPy array

    # Calcular bbox usando la función de OpenCV
    bbox = get_bbox_from_image_array(img_np)

    return img_np, bbox

def generate_variations_from_base(base_image_np, bbox):
    """
    Genera las 5 variaciones adicionales de una imagen de caracter
    a partir de una imagen base (caracter blanco sobre fondo negro).
    """
    variations = []

    # --- Variaciones con blur ---
    img_blur_black = base_image_np.copy()
    img_blur_black = apply_gaussian_blur(img_blur_black)
    variations.append((img_blur_black.copy(), "blur", "blanco"))


    # --- Variaciones con rotacion ---
    img_rotate_black_1 = base_image_np.copy()
    img_rotate_black_1 = rotate_image(img_rotate_black_1, 5, (0, 0, 0))
    img_rotate_black_1 = cv2.resize(img_rotate_black_1, (base_image_np.shape[1], base_image_np.shape[0]))
    variations.append((img_rotate_black_1, "rotate", "blanco"))
    img_rotate_black_2 = base_image_np.copy()
    img_rotate_black_2 = rotate_image(img_rotate_black_2, -5, (0, 0, 0))
    img_rotate_black_2 = cv2.resize(img_rotate_black_2, (base_image_np.shape[1], base_image_np.shape[0]))
    variations.append((img_rotate_black_2, "rotate", "blanco"))

    return variations

In [21]:
# --- Función para generar los datasets principales ---
def generate_full_datasets(font_df):
    """
    Genera el dataset de NumPy y Pandas a partir de las fuentes TTF.

    Args:
        font_df: dataset de fuentes.

    Returns:
        tuple: (numpy_dataset, pandas_dataset)
    """

    numpy_data_list = []
    pandas_data = []
    global_index = 0

    # Caracteres a generar
    characters_to_generate = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"

    if font_df.empty:
        print("No se encontraron fuentes para procesar.")
        return np.array([], dtype=[('image', object), ('bbox', object), ('index', int)]), pd.DataFrame()

    for index, font_row in font_df.iterrows():
        font_name = font_row['font_name']
        font_path = font_row['font_path']
        print(f"Generando caracteres para la fuente: {font_name} ({font_path})")

        for char in characters_to_generate:
            # Generar la imagen base (carácter blanco sobre fondo negro) y su bbox con Pillow
            base_image_np, bbox = generate_base_char_image_and_bbox(char, font_path)

            if base_image_np is None or bbox is None:
                print(f"Saltando '{char}' de '{font_name}' debido a un error en la generación de la imagen base o bbox.")
                continue

            # La imagen base se añade al dataset de NumPy
            numpy_data_list.append((base_image_np, bbox, global_index))
            pandas_data.append({
                'index': global_index,
                'character': char,
                'font_type': font_name,
                'background_type': "negro", # La base siempre es blanco sobre negro
                'character_color': "blanco"
            })
            global_index += 1

            # Generar las otras 5 variaciones usando OpenCV/NumPy
            other_variations = generate_variations_from_base(base_image_np, bbox)

            for img_var, bg_type, char_color in other_variations:
                if bg_type == 'rotate' and char_color == 'blanco':
                    bbox = get_bbox_from_image_array(img_var)
                elif bg_type == 'rotate' and char_color == 'negro':
                    bbox = get_bbox_from_image_array(cv2.bitwise_not(img_var.copy()))
                numpy_data_list.append((img_var, bbox, global_index))
                pandas_data.append({
                    'index': global_index,
                    'character': char,
                    'font_type': font_name,
                    'background_type': bg_type,
                    'character_color': char_color
                })
                global_index += 1

    numpy_dataset = np.array(numpy_data_list, dtype=[('image', object), ('bbox', object), ('index', int)])
    pandas_dataset = pd.DataFrame(pandas_data)

    return numpy_dataset, pandas_dataset

In [22]:
# Generar los datasets
font_df = create_font_dataset('c:/tmp/content/ocr_fonts_latin')
numpy_dataset, pandas_dataset = generate_full_datasets(font_df)

print("\nGeneración de datasets completada.")
print("\nDataset de NumPy (estructura):")
print(numpy_dataset.dtype)
print(f"Número total de entradas en NumPy dataset: {len(numpy_dataset)}")

print("\nDataset de Pandas (primeras 5 filas):")
print(pandas_dataset.head())
print(f"Número total de entradas en Pandas dataset: {len(pandas_dataset)}")

Buscando archivos .ttf en: c:/tmp/content/ocr_fonts_latin
Generando caracteres para la fuente: ABeeZee (c:/tmp/content/ocr_fonts_latin\ABeeZee\ABeeZee-italic.ttf)
Generando caracteres para la fuente: Abhaya_Libre (c:/tmp/content/ocr_fonts_latin\Abhaya_Libre\Abhaya_Libre-500.ttf)
Generando caracteres para la fuente: Aboreto (c:/tmp/content/ocr_fonts_latin\Aboreto\Aboreto-regular.ttf)
Generando caracteres para la fuente: Abril_Fatface (c:/tmp/content/ocr_fonts_latin\Abril_Fatface\Abril_Fatface-regular.ttf)
Generando caracteres para la fuente: Abyssinica_SIL (c:/tmp/content/ocr_fonts_latin\Abyssinica_SIL\Abyssinica_SIL-regular.ttf)
Generando caracteres para la fuente: Aclonica (c:/tmp/content/ocr_fonts_latin\Aclonica\Aclonica-regular.ttf)
Generando caracteres para la fuente: ADLaM_Display (c:/tmp/content/ocr_fonts_latin\ADLaM_Display\ADLaM_Display-regular.ttf)
Generando caracteres para la fuente: Advent_Pro (c:/tmp/content/ocr_fonts_latin\Advent_Pro\Advent_Pro-100.ttf)
Generando caractere

In [3]:
muestra = pandas_dataset.sample(10)
for index, row in muestra.iterrows():
    print(f"Index: {row['index']}, Character: {row['character']}, Font Type: {row['font_type']}, Background Type: {row['background_type']}, Character Color: {row['character_color']}")
    img_muestra = numpy_dataset[row['index']]['image']
    bbox= numpy_dataset[row['index']]['bbox']
    cv2.rectangle(img_muestra, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (128, 128, 128), 1)
    plt.imshow(img_muestra, cmap='gray')
    plt.show()
    print("\n")

NameError: name 'pandas_dataset' is not defined

## Dataset de entrenamiento
Este dataset contendra imagenes de carcateres y las coordenadas de la caja que los contiene

In [ ]:
#composed_numpy_dataset = np.load('c:/tmp/content/ocr_composed_images_dataset.npy', allow_pickle=True)

In [24]:
composed_numpy_dataset = numpy_dataset
composed_pandas_dataset = pandas_dataset

## Generacion de archivos de entrenamiento

In [25]:
# Configuración del archivo data.yaml para YOLOv8
characters_to_generate = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
class_id_to_char = {i: char for i, char in enumerate(characters_to_generate)}
names_string = "["
for char in characters_to_generate:
    if char == characters_to_generate[-1]:
        names_string += f"'{char}'"
    else:
        names_string += f"'{char}', "
names_string += "]"

# Configuración del archivo data.yaml para YOLOv8
with open("data.yaml", "w", encoding="utf-8") as f:
    f.write(f"""
path: c:/tmp/content/training
train: images/train
val: images/val
names: {names_string}
""")

# Generar el diccionario propio con ID y letra
class_id_to_char = {i: char for i, char in enumerate(characters_to_generate)}


def get_id_by_char(char):
    """
    Busca el ID de clase dado un carácter, iterando directamente sobre el diccionario.
    Devuelve el ID si se encuentra, None si el carácter no está en las clases.
    """
    for class_id, stored_char in class_id_to_char.items():
        if stored_char == char:
            return class_id
    return None # Si el carácter no se encontró después de buscar en todo el diccionario

def get_char_by_id(class_id):
    return class_id_to_char.get(class_id)

In [27]:
os.makedirs('c:/tmp/content/training/images/train', exist_ok=True)
os.makedirs('c:/tmp/content/training/images/val', exist_ok=True)
os.makedirs('c:/tmp/content/training/labels/train', exist_ok=True)
os.makedirs('c:/tmp/content/training/labels/val', exist_ok=True)

# Obtener el número total de muestras
num_samples = len(composed_numpy_dataset['image'])
indices = list(range(num_samples)) # Generar índices de 0 a num_samples-1
train_idx, val_idx = train_test_split(indices, test_size=0.1, random_state=42)

def save_image_and_label(idx, image, labels_list, bboxes_list, split):
    """
    Guarda una imagen y su archivo de etiquetas en formato YOLO.

    Args:
        idx (int): Índice de la imagen (usado para el nombre del archivo).
        image (np.array): Datos de la imagen (en escala de grises, asumido 0-1 o 0-255).
        bboxes_list (list): Lista de bounding boxes para la imagen.
                            Cada bbox es una lista/tupla [x_min, y_min, x_max, y_max].
                            Puede estar vacía si no hay caracteres.
        split (str): 'train' o 'val', para determinar el directorio de destino.
    """
    img_path = f'c:/tmp/content/training/images/{split}/{idx}.jpg'
    label_path = f'c:/tmp/content/training/labels/{split}/{idx}.txt'


    # Obtener las dimensiones de la imagen guardada para la normalización
    h, w = image.shape # Asume que la imagen es 2D (escala de grises)
    img_to_save = Image.fromarray((image * 255).astype('uint8')).convert('L')
    img_to_save.save(img_path)

    # Abre el archivo de etiquetas para escribir las coordenadas de todos los bboxes
    with open(label_path, 'w') as f:
        for label, bbox in zip(labels_list, bboxes_list):
            x_min, y_min, x_max, y_max = bbox

            # Formato YOLO: class x_center y_center width height (normalizado)
            xc = (x_min + x_max) / 2 / w
            yc = (y_min + y_max) / 2 / h
            bw = (x_max - x_min) / w
            bh = (y_max - y_min) / h

            label_id = get_id_by_char(label)
            if label_id is None:
                print(f"Advertencia: El carácter '{label}' no se encontró en las clases. Saltando.")
                continue

            f.write(f'{label_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n')


# Bucle principal para procesar y guardar imágenes y etiquetas
# CAMBIO PRINCIPAL: Iteración sobre los índices directamente y acceso al dataset NumPy
total = len(indices)
contador = 1
for idx in indices: # Iteramos sobre la lista de índices generada
    split = 'train' if idx in train_idx else 'val'
    if (contador * 100) % total == 0:
        print(f"{contador * 100 / total}% completado.")
    contador += 1

    # Accedemos a la imagen y su lista de bboxes usando el índice actual
    current_image = composed_numpy_dataset['image'][idx]
    current_bboxes = [composed_numpy_dataset['bbox'][idx]]
    index = composed_numpy_dataset['index'][idx]
    current_data = composed_pandas_dataset[composed_pandas_dataset['index'] == index].iloc[0]
    if current_data.empty:
        print(f"Advertencia: No se encontraron datos para el índice {index}. Saltando.")
    current_labels = [current_data['character']]

    save_image_and_label(idx, current_image, current_labels, current_bboxes, split)

25.0% completado.
50.0% completado.
75.0% completado.
100.0% completado.


In [31]:
import os

# Asegúrate de que 'characters_to_generate' sea exactamente la misma cadena
# que utilizaste para generar tu data.yaml
characters_to_generate = "ABCDEFGHIJKLMNOPQRSTUVWXYZÑabcdefghijklmnopqrstuvwxyzñáéíóúü0123456789.,;:?¿!¡-—&*/+=-_@#$%"
num_classes = len(characters_to_generate)
max_valid_id = num_classes - 1

# Asegúrate de que estas rutas son correctas y accesibles en tu sistema
label_dirs = [
    'c:/tmp/content/training/labels/train',
    'c:/tmp/content/training/labels/val'
]

print(f"Verificando etiquetas. Número de clases esperado: {num_classes} (IDs 0 a {max_valid_id})")

errors_found = False

for label_dir in label_dirs:
    print(f"\n--- Procesando directorio: {label_dir} ---")
    if not os.path.exists(label_dir):
        print(f"Advertencia: Directorio no encontrado: {label_dir}. Omite este directorio.")
        continue

    txt_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    if not txt_files:
        print(f"No se encontraron archivos .txt en {label_dir}.")
        continue

    for label_file in txt_files:
        file_path = os.path.join(label_dir, label_file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f: # Añadir encoding='utf-8' por si hay caracteres especiales en las rutas o nombres
                for line_num, line in enumerate(f):
                    parts = line.strip().split()

                    # 1. Verificar si la línea está vacía
                    if not parts:
                        continue # Ignorar líneas completamente vacías

                    # 2. Verificar la cantidad correcta de valores
                    if len(parts) != 5:
                        print(f"ERROR: Cantidad incorrecta de valores en {label_file} línea {line_num+1}.")
                        print(f"  Contenido: '{line.strip()}' (Esperado: 5 valores: class_id cx cy w h)")
                        errors_found = True
                        continue # Pasa a la siguiente línea

                    try:
                        class_id = int(parts[0])

                        # 3. Verificar el rango del ID de clase
                        if not (0 <= class_id <= max_valid_id):
                            print(f"ERROR: ID de clase '{class_id}' fuera de rango en {label_file} línea {line_num+1}.")
                            print(f"  Rango válido para ID de clase: 0 a {max_valid_id}.")
                            errors_found = True

                        # 4. Verificar el rango de las coordenadas
                        # Las coordenadas son cx, cy, w, h (índices 1 a 4)
                        for i in range(1, 5):
                            val_str = parts[i]
                            val = float(val_str)
                            if not (0.0 <= val <= 1.0):
                                print(f"ERROR: Coordenada '{val_str}' fuera de rango [0.0, 1.0] en {label_file} línea {line_num+1} (posición {i}).")
                                errors_found = True
                            # Opcional: También podrías querer verificar si width o height son 0 o extremadamente pequeños
                            if i in [3, 4] and val < 0.0001: # Check for width/height being too small
                                print(f"ADVERTENCIA: Ancho/Alto muy pequeño ({val_str}) en {label_file} línea {line_num+1} (posición {i}).")

                    except ValueError:
                        print(f"ERROR: Error de tipo de dato (no numérico) en {label_file} línea {line_num+1}.")
                        print(f"  Contenido: '{line.strip()}'")
                        errors_found = True
                    except IndexError: # En caso de que split() devuelva menos elementos de lo esperado inesperadamente
                        print(f"ERROR: Error de índice al procesar línea {line_num+1} en {label_file}.")
                        print(f"  Contenido: '{line.strip()}'")
                        errors_found = True

        except FileNotFoundError:
            print(f"ERROR: Archivo no encontrado: {file_path}")
            errors_found = True
        except Exception as e:
            print(f"ERROR: Error inesperado al leer el archivo {file_path}: {e}")
            errors_found = True

print("\n--- Verificación de etiquetas completada ---")
if errors_found:
    print("¡Se encontraron errores! Por favor, revisa los mensajes de ERROR y corrige tus archivos de etiquetas.")
    print("Recuerda usar 'CUDA_LAUNCH_BLOCKING=1' para depurar el entrenamiento de YOLOv8 si el problema persiste.")
else:
    print("¡No se encontraron errores ni advertencias importantes en las etiquetas!")

Verificando etiquetas. Número de clases esperado: 91 (IDs 0 a 90)

--- Procesando directorio: c:/tmp/content/training/labels/train ---

--- Procesando directorio: c:/tmp/content/training/labels/val ---

--- Verificación de etiquetas completada ---
¡No se encontraron errores ni advertencias importantes en las etiquetas!


## Entrenamiento del Modelo YOLO 8

In [4]:
model = YOLO('yolov8n.pt')
model.train(
    data='data.yaml',
    epochs=10,
    imgsz=256,
    batch=64,
    workers=8,
    show=False,
    augment=False
)

print("Entrenamiento del modelo YOLOv8 completado.")

New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.155  Python-3.9.23 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning C:\tmp\content\training\labels\train... 295822 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29582


train: New cache created: C:\tmp\content\training\labels\train.cache
val: Fast image access  (ping: 0.20.2 ms, read: 5.31.9 MB/s, size: 3.9 KB)


val: Scanning C:\tmp\content\training\labels\val... 32870 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32870/3287


val: New cache created: C:\tmp\content\training\labels\val.cache
Plotting labels to runs\detect\train12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs\detect\train12
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all      32870      32870      0.926      0.944      0.965      0.958

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      2.02G     0.1416     0.2682     0.7685         14        256: 100%|██████████| 4623/4623 [20:58<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 257/257 [03:


                   all      32870      32870      0.929      0.948      0.968      0.962

10 epochs completed in 4.121 hours.
Optimizer stripped from runs\detect\train12\weights\last.pt, 6.7MB
Optimizer stripped from runs\detect\train12\weights\best.pt, 6.7MB

Validating runs\detect\train12\weights\best.pt...
Ultralytics 8.3.155  Python-3.9.23 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 72 layers, 3,252,785 parameters, 0 gradients, 9.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 257/257 [03:


                   all      32870      32870       0.93      0.948      0.968      0.962
                     A        359        359      0.959      0.994      0.987      0.986
                     B        375        375      0.921      0.981      0.979      0.977
                     C        387        387      0.948      0.891      0.972      0.971
                     D        372        372      0.928      0.995      0.986      0.985
                     E        382        382      0.924      0.992       0.97      0.968
                     F        393        393      0.939       0.98      0.983      0.978
                     G        392        392      0.934      0.982      0.977      0.977
                     H        365        365      0.927      0.962      0.979      0.978
                     I        393        393      0.781      0.783      0.881      0.861
                     J        340        340      0.849      0.959      0.962       0.96
                     

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# --- Configura la ruta de tu directorio de resultados ---
# Puedes encontrar el nombre de la carpeta de la última ejecución en la salida de tu entrenamiento.
# Por ejemplo, si tu entrenamiento más reciente guardó los resultados en 'runs/detect/train', usa eso.
# Si estás ejecutando esto inmediatamente después de entrenar, el directorio 'train' será el más reciente.
# Si hay múltiples 'trainX' carpetas, ajusta 'run_dir_name' a la correcta.
# Por ejemplo, si el último fue 'train3', sería 'train3'.
run_dir_name = 'train10' # O 'train2', 'train3', etc., según tu última ejecución.
results_path = f'C:/Users/Andres/runs/detect/{run_dir_name}/results.csv'

# Comprueba si el archivo existe antes de intentar leerlo
if not os.path.exists(results_path):
    print(f"Error: El archivo de resultados no se encontró en '{results_path}'.")
    print("Asegúrate de que la ruta sea correcta y que el entrenamiento haya finalizado.")
    print("Revisa los nombres de las carpetas en 'c:/tmp/content/runs/detect/' para encontrar la correcta.")
else:
    # Cargar los resultados en un DataFrame de pandas
    # Los resultados.csv de YOLOv8 tienen la cabecera comentada con #, así que la omitimos.
    # También hay un espacio extra antes de los nombres de columna, lo removemos.
    df_results = pd.read_csv(results_path, skipinitialspace=True)

    # Limpiar los nombres de las columnas para facilitar el acceso
    # Las columnas suelen ser como '                 metrics/precision(B)'
    df_results.columns = df_results.columns.str.strip()

    print("Métricas de entrenamiento completadas:")
    # Muestra las últimas filas del DataFrame para ver las métricas finales
    print(df_results.tail())

Métricas de entrenamiento completadas:
   epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
5      6  348.149         0.62970         0.33365         0.78549   
6      7  400.771         0.60112         0.31655         0.78390   
7      8  453.348         0.57391         0.30118         0.78204   
8      9  509.248         0.55311         0.28790         0.78068   
9     10  563.207         0.53119         0.27323         0.77936   

   metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
5               0.97232            0.84508           0.92622   
6               0.97262            0.85288           0.92461   
7               0.97651            0.86268           0.94055   
8               0.97040            0.84788           0.92935   
9               0.97457            0.85230           0.93070   

   metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
5              0.79039       0.58643       0.33194       0.78342  0.001010   
6    

In [ ]:
   # --- Visualización de las métricas (opcional, pero muy útil) ---
if os.path.exists(results_path):
# --- Visualización de las métricas (opcional, pero muy útil) ---
    fig1 = plt.figure(figsize=(12, 6)) # Asigna la figura a una variable

    # Graficar mAP@0.50
    if 'metrics/mAP50(B)' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['metrics/mAP50(B)'], label='mAP@0.50', marker='o', markersize=4)
    else:
        print("La columna 'metrics/mAP50(B)' no se encontró en los resultados.")

    # Graficar mAP@0.50-0.95
    if 'metrics/mAP50-95(B)' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['metrics/mAP50-95(B)'], label='mAP@0.50-0.95', marker='x', markersize=4)
    else:
        print("La columna 'metrics/mAP50-95(B)' no se encontró en los resultados.")

    plt.title('Curva de mAP durante el Entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('mAP')
    plt.grid(True)
    plt.legend()
    plt.show() # Intenta mostrar el gráfico

    # Guarda la primera figura después de intentar mostrarla
    save_path_map = os.path.join(os.path.dirname(results_path), 'map_curve.png')
    fig1.savefig(save_path_map)
    print(f"Gráfico de mAP guardado en: {save_path_map}")
    plt.close(fig1) # Cierra la figura para liberar memoria


    fig2 = plt.figure(figsize=(12, 6)) # Asigna la segunda figura a una variable
    # Graficar la pérdida de entrenamiento y validación
    if 'train/box_loss' in df_results.columns and 'val/box_loss' in df_results.columns:
        plt.plot(df_results['epoch'], df_results['train/box_loss'], label='Pérdida de Bounding Box (Entrenamiento)')
        plt.plot(df_results['epoch'], df_results['val/box_loss'], label='Pérdida de Bounding Box (Validación)')
    else:
        print("Las columnas de pérdida ('train/box_loss' o 'val/box_loss') no se encontraron en los resultados.")

    plt.title('Curvas de Pérdida durante el Entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.grid(True)
    plt.legend()
    plt.show() # Intenta mostrar el segundo gráfico

    # Guarda la segunda figura después de intentar mostrarla
    save_path_loss = os.path.join(os.path.dirname(results_path), 'loss_curve.png')
    fig2.savefig(save_path_loss)
    print(f"Gráfico de pérdidas guardado en: {save_path_loss}")
    plt.close(fig2) # Cierra la figura


<Figure size 1200x600 with 1 Axes>

Gráfico de mAP guardado en: C:/Users/Andres/runs/detect/train10\map_curve.png


<Figure size 1200x600 with 1 Axes>

Gráfico de pérdidas guardado en: C:/Users/Andres/runs/detect/train10\loss_curve.png
